In [ ]:
import pandas as pd
df=pd.read_csv('2022.csv')

In [ ]:
df.head()

In [ ]:
import numpy as np
df.replace(np.nan, 0)


In [ ]:
df=df.drop(['CommitmentDescription '],axis=1)
df

In [ ]:
df.drop(df.tail(1).index,inplace=True) 

In [ ]:
df

In [ ]:
df.replace(np.nan, 0)

In [ ]:
# Determine the number of unique values in each column.
cat = df.dtypes[df.dtypes == "object"].index.tolist()
df[cat].nunique()

In [ ]:
# Look at Commitment# value counts for binning
type_counts = df.Commitment.value_counts()
type_counts

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

In [ ]:
# Visualize the value counts of TYPE
type_counts.plot.density()

In [ ]:
# Determine which values to replace if counts are less than ...?
replace_application = list(type_counts[type_counts < 30 ].index)

# Replace in dataframe
for app in replace_application:
    df.Commitment = df.Commitment.replace(app,"Other")
    
# Check to make sure binning was successful
df.Commitment.value_counts()
                         


In [ ]:
# Look at CLASSIFICATION value counts for binning
acct_counts = df.AccountName.value_counts()
acct_counts


In [ ]:
acct_counts.plot.density()

In [ ]:
# Determine which values to replace if counts are less than ..?
replace_class = list(acct_counts[acct_counts < 30].index)

# Replace in dataframe
for cls in replace_class:
    df.AccountName = df.AccountName.replace(cls,"Other")
    
# Check to make sure binning was successful
df.AccountName.value_counts()

In [ ]:
# Generate our categorical variable lists
app_cat = df.dtypes[df.dtypes == "object"].index.tolist()
df[app_cat].nunique()

In [ ]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(df[app_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(app_cat)
encode_df.head()

In [ ]:
# Merge one-hot encoded features and drop the originals
new_df = df.merge(encode_df,left_index=True, right_index=True)
new_df = new_df.drop(app_cat,1)
new_df.head()

In [ ]:
new_df=new_df.replace(np.nan, 0)

In [ ]:
new_df


In [ ]:
y =new_df["OverBudget "].values
X = new_df.drop(columns=["OverBudget ","MeetTarget"]).values
# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [ ]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder

import tensorflow as tf

In [ ]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=78)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")

In [ ]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 24
hidden_nodes_layer2 = 12


nn = tf.keras.models.Sequential()

In [ ]:
# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

In [ ]:
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

In [ ]:
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")